In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
import random 
import pandas as pd

from sktime.datatypes._panel._convert import from_3d_numpy_to_nested
from sklearn.pipeline import Pipeline

from sktime.classification.compose import ColumnEnsembleClassifier
from sktime.datasets import load_arrow_head
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sktime.transformations.panel.compose import ColumnConcatenator
from sktime.classification.shapelet_based import MrSEQLClassifier
from sktime.datasets import load_basic_motions
from sklearn.impute import KNNImputer

In [2]:
# TA CODE
def load_data(data_path):
    
    FEATURES = range(2, 33)
    N_TIME_SERIES = 3500

    # Create the training and testing samples
    LS_path = os.path.join(data_path, 'LS')
    TS_path = os.path.join(data_path, 'TS')
    X_train, X_test = [np.zeros((N_TIME_SERIES, len(FEATURES), 512)) for i in range(2)]
    
    for f in FEATURES:
        data = np.loadtxt(os.path.join(LS_path, 'LS_sensor_{}.txt'.format(f)))
        for i in range(N_TIME_SERIES):
            X_train[i][f-2][:] = data[i]
        data = np.loadtxt(os.path.join(TS_path, 'TS_sensor_{}.txt'.format(f)))
        for i in range(N_TIME_SERIES):
            X_test[i][f-2][:] = data[i]
            
    y_train = np.loadtxt(os.path.join(LS_path, 'activity_Id.txt'))
       
    return X_train, y_train, X_test

def write_submission(y, where, submission_name='toy_submission.csv'):
    os.makedirs(where, exist_ok=True)

    SUBMISSION_PATH = os.path.join(where, submission_name)
    if os.path.exists(SUBMISSION_PATH):
        os.remove(SUBMISSION_PATH)

    y = y.astype(int)
    outputs = np.unique(y)

    # Verify conditions on the predictions
    if np.max(outputs) > 14:
        raise ValueError('Class {} does not exist.'.format(np.max(outputs)))
    if np.min(outputs) < 1:
        raise ValueError('Class {} does not exist.'.format(np.min(outputs)))
    
    # Write submission file
    with open(SUBMISSION_PATH, 'a') as file:
        n_samples = len(y)
        if n_samples != 3500:
            raise ValueError('Check the number of predicted values.')

        file.write('Id,Prediction\n')

        for n, i in enumerate(y):
            file.write('{},{}\n'.format(n+1, int(i)))

In [4]:
if __name__ == '__main__':
    # Directory containing the data folders
    DATA_PATH = 'data'
    X_train, y_train, X_test = load_data(DATA_PATH)

In [5]:
    # Replace missing values
    X_train = from_3d_numpy_to_nested(X_train)
    X_test = from_3d_numpy_to_nested(X_test)
    print(X_train.shape)

(3500, 31)


In [ ]:
    # clf = TimeSeriesForestClassifier(n_jobs = -1, random_state = 0).fit(X_train, y_train)
    # X, y = load_arrow_head(return_X_y=True)
    # print(X)
    steps = [
    ("concatenate", ColumnConcatenator()),
    ("classify", TimeSeriesForestClassifier(n_estimators=100, n_jobs = -1)),
    ]
    
    clf = Pipeline(steps)
    clf.fit(X_train, y_train)
    clf.score(X_test, y_test)
#     LS_path = os.path.join(DATA_PATH, 'LS')   
#     LS_subject_id = np.loadtxt(os.path.join(LS_path, 'subject_Id.txt'))
    
#     ids = [1, 2, 3, 4, 5]
#     learning_id = [0, 0, 0]   
#     scores = np.zeros(10)
    
#     random.seed()
    
#     for j in range(10):
#         random.shuffle(ids)
        
#         for i in range(3):
#             learning_id[i] = ids[i]
            
#         unique_ls, count_ls = np.unique(LS_subject_id, return_counts = True)
        
#         count = np.asarray((unique_ls, count_ls))
        
#         training_size = int(count[1][learning_id[0] - 1] + count[1][learning_id[1] - 1] + count[1][learning_id[2] - 1])
        
#         X_train_split = np.zeros((training_size, 31))
#         X_test_split = np.zeros((3500 - training_size, 31))

#         y_train_split = np.zeros((training_size, 1))
#         y_test_split = np.zeros((3500 - training_size, 1))

#         training_current_size, testing_current_size = 0, 0

#         for i in range(3500):
#             if LS_subject_id[i] in learning_id:
#                 X_train_split[training_current_size] = X_train[i]
#                 y_train_split[training_current_size] = y_train[i]
#                 training_current_size += 1
#             else:
#                 X_test_split[testing_current_size] = X_train[i]
#                 y_test_split[testing_current_size] = y_train[i]
#                 testing_current_size += 1
                
#         clf = TimeSeriesForestClassifier(n_jobs = -1, random_state = 0).fit(X_train_split, y_train_split)
#         y_pred = clf.predict(X_test_split)
#         scores[i] = accuracy_score(y_test_split, y_pred)
    
#     print(np.mean(scores))

In [62]:
    clf.fit(X_train, y_train)
    y_test = clf.predict(X_test)
    write_submission(y_test, 'submissions')

/home/olivierjoris/miniconda3/envs/ELEN0062/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:619: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


KeyboardInterrupt: 